# Text2SQL & AI Search with Semantic Kernel & Azure OpenAI

This notebook demonstrates how the SQL plugin can be integrated with Semantic Kernel and Azure OpenAI to answer questions from the database based on the schemas provided. Additionally, it integrates with an AI Search plugin to show how both can be used in parallel to answer questions based on the best possible source. The prompt may need to be tweaked for your usage.

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

In [ ]:
import logging
import os
import yaml
import dotenv
import json
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from plugins.sql_plugin.sql_plugin import SQLPlugin
from plugins.ai_search_plugin.ai_search_plugin import AISearchPlugin
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from IPython.display import display, Markdown

logging.basicConfig(level=logging.INFO)

## Kernel Setup

In [ ]:
dotenv.load_dotenv()
kernel = Kernel()

## Set up GPT connections

In [ ]:
service_id = "chat"

In [ ]:
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name=os.environ["OPEN_AI_CONVERSATION_MODEL"],
    endpoint=os.environ["OPEN_AI_ENDPOINT"],
    api_key=os.environ["OPEN_AI_KEY"],
)
kernel.add_service(chat_service)

In [ ]:
# Register the SQL Plugin with the Database name to use.
sql_plugin = SQLPlugin(database=os.environ["SQL_DB_NAME"])
kernel.add_plugin(sql_plugin, "SQL")

In [ ]:
ai_search_plugin = AISearchPlugin()
kernel.add_plugin(ai_search_plugin, "AISearch")

## Prompt Setup

In [ ]:
# Load prompt and execution settings from the file
with open("./prompt.yaml", "r") as file:
    data = yaml.safe_load(file.read())
    prompt_template_config = PromptTemplateConfig(**data)

In [ ]:
chat_function = kernel.add_function(
    prompt_template_config=prompt_template_config,
    plugin_name="ChatBot",
    function_name="Chat",
)

## ChatBot setup

In [ ]:
history = ChatHistory()

In [ ]:
async def ask_question(question: str, chat_history: ChatHistory) -> str:
    """Asks a question to the chatbot and returns the answer.
    
    Args:
        question (str): The question to ask the chatbot.
        chat_history (ChatHistory): The chat history object.
        
    Returns:
        str: The answer from the chatbot.
    """

    # Create important information prompt that contains the SQL database information and the AI search description (useful if you have multiple indexes).
    engine_specific_rules = "Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error."
    important_information_prompt = f"""
    [AI SEARCH INFORMATION]
    {ai_search_plugin.system_prompt()}
    [END AI SEARCH INFORMATION]

    [SQL DATABASE INFORMATION]
    {sql_plugin.system_prompt(engine_specific_rules=engine_specific_rules)}
    [END SQL DATABASE INFORMATION]
    """

    arguments = KernelArguments()
    arguments["chat_history"] = chat_history
    arguments["important_information"] = important_information_prompt
    arguments["user_input"] = question

    logging.info("Question: %s", question)

    answer = await kernel.invoke(
        function_name="Chat",
        plugin_name="ChatBot",
        arguments=arguments,
    )

    logging.info("Answer: %s", answer)

    # Log the question and answer to the chat history.
    chat_history.add_user_message(question)
    chat_history.add_message({"role": "assistant", "message": answer})

    json_answer = json.loads(str(answer))

    display(Markdown(json_answer["answer"]))